# Potential Bucket Enumeration on AWS

Looks for potential enumeration of AWS buckets via ListBuckets.

References:
* https://github.com/Lifka/hacking-resources/blob/c2ae355d381bd0c9f0b32c4ead049f44e5b1573f/cloud-hacking-cheat-sheets.md
* https://jamesonhacking.blogspot.com/2020/12/pivoting-to-private-aws-s3-buckets.html
* https://securitycafe.ro/2022/12/14/aws-enumeration-part-ii-practical-enumeration/

False Positives:
* Administrators listing buckets, it may be necessary to filter out users who commonly conduct this activity.

## Detection Query
Execute the following query to find alerts related to this rule.

In [ ]:
%ingest.source_type="aws:cloudtrail"
eventSource="s3.amazonaws.com"
eventName="ListBuckets"
(not userIdentity.type="AssumedRole")
| stats
min(@scnr.datetime) as firstTime,
max(@scnr.datetime) as lastTime,
count() as eventCount
by
userIdentity.arn,
eventSource,
eventName,
awsRegion

## Investigation Steps
1. **Identify the Principal**: Who performed the action?
2. **Review Context**: What else did this principal do?
3. **Check Permissions**: Did the principal have legitimate reasons for this action?
4. **Verify Resource**: Is the resource critical or sensitive?

In [ ]:
%ingest.source_type:aws:cloudtrail
userIdentity.arn: <PRINCIPAL_ARN>
| count(eventName) by eventName